In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F
from PIL import Image

In [2]:
transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
])
train_data = ImageFolder(root="./CarsDataset/train", transform=transform)
test_data = ImageFolder(root="./CarsDataset/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [3]:
class CNN(nn.Module):
    def __init__(self, num_classes, seed = 155):
        super(CNN, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn4 = nn.BatchNorm2d(128)
    
        self.fc1 = nn.Linear(10*10*128, 512) 
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x): #el input de cada uno es el input de la conv, por el tamaño de la imagen, y el output lo mismo. El tamaño de la imagen se reduce en kernel-stride
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
    
        x = x.view(x.size(0), -1)
    
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def predict_image(self, image_path):
        image = transform(Image.open(image_path)).unsqueeze(0)

        with torch.no_grad():
            output = self(image)

        probabilities = torch.softmax(output, dim=1)[0]
        predicted_label = torch.argmax(probabilities).item()

        class_names = train_data.classes
        predicted_class = class_names[predicted_label]

        print(f'Predicted Class: {predicted_class}')
        for i, prob in enumerate(probabilities):
            print(f'{class_names[i]}: {prob:.4f}')

model = CNN(num_classes=len(train_data.classes))

In [10]:
learning_rate = 0.008
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Entrenamiento del modelo
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")

print('Finished Training')

Epoch 1, Training Loss: 1.3647247320129758
Epoch 2, Training Loss: 1.266402945064363
Epoch 3, Training Loss: 1.226705729393732
Epoch 4, Training Loss: 1.1335751840046473
Epoch 5, Training Loss: 1.0688737818173
Epoch 6, Training Loss: 0.9760117950893584
Epoch 7, Training Loss: 0.8829589937414442
Epoch 8, Training Loss: 0.8004206203279041
Epoch 9, Training Loss: 0.7383607225758689
Epoch 10, Training Loss: 0.7135690655027117
Finished Training


In [6]:
# Evaluación del modelo en el conjunto de prueba
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Accuracy on test set: {100 * correct / total}%")

Accuracy on test set: 43.17343173431734%


In [7]:
# Ejemplo de predicción de una imagen
image_path = "./CarsDataset/PruebaRollsRoyce.jpg"
model.predict_image(image_path)
image_path = "./CarsDataset/PruebaMahindraScorpio.jpg"
model.predict_image(image_path)
image_path = "./CarsDataset/PruebaMahindraScorpio2.png"
model.predict_image(image_path)
image_path = "./CarsDataset/Audi.jpg"
model.predict_image(image_path)

Predicted Class: Audi
Audi: 0.4317
Hyundai Creta: 0.0296
Mahindra Scorpio: 0.0272
Rolls Royce: 0.1352
Swift: 0.0617
Tata Safari: 0.2850
Toyota Innova: 0.0296
Predicted Class: Toyota Innova
Audi: 0.1883
Hyundai Creta: 0.1236
Mahindra Scorpio: 0.1363
Rolls Royce: 0.1021
Swift: 0.1089
Tata Safari: 0.0836
Toyota Innova: 0.2573
Predicted Class: Audi
Audi: 0.4677
Hyundai Creta: 0.0176
Mahindra Scorpio: 0.0148
Rolls Royce: 0.1254
Swift: 0.0458
Tata Safari: 0.3110
Toyota Innova: 0.0177
Predicted Class: Audi
Audi: 0.5129
Hyundai Creta: 0.0059
Mahindra Scorpio: 0.0042
Rolls Royce: 0.1011
Swift: 0.0239
Tata Safari: 0.3461
Toyota Innova: 0.0060
